In [29]:
import sklearn
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from PyRadioLoc.Utils.GeoUtils import GeoUtils
from sklearn.ensemble import RandomForestRegressor
%run Imports.ipynb

def write_to_csv(x_pred, y_pred, file_path):
    #labels = ["rssi_1_1", "rssi_1_2", "rssi_1_3", "rssi_2_1", "rssi_2_2", "rssi_2_3", "rssi_3_1", "rssi_3_2", "rssi_3_3"]
    #y_pred_dataframe = pd.DataFrame(y_pred)
    res = pd.concat([x_pred, y_pred], axis=1)
    res.to_csv(path_or_buf=file_path, index = False)

In [2]:
db_train = pd.read_csv("databases/Outdoor_X_train_90%.csv")
X_train = db_train.drop("idx", axis = 1)
X_train = X_train.drop("lat", axis = 1)
X_train = X_train.drop("lon", axis = 1)
#lon_train = db_train["lon"]
#lat_train = db_train["lat"]
#X_train = pd.concat([lat_train, lon_train], axis=1)
X_train.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delta_1_12,...,ang_3,cos_3,sin_3,tg_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23
0,-91.801429,-92.775714,-93.930000,-96.246667,-92.3700,-86.497143,-91.915000,-83.340000,-95.8575,0.974286,...,111.046648,-0.359128,0.933288,-2.598763,7,3,4,4,3,-1
1,-87.827143,-90.397143,-88.951429,-67.090000,-82.1725,-81.895714,-86.488333,-92.622000,-97.6000,2.570000,...,71.008261,0.325432,0.945566,2.905572,4,2,5,2,-1,-3
2,-96.500000,-89.047013,-94.131524,-95.600000,-94.3100,-74.561071,-88.900000,-91.629273,-95.8575,-7.452987,...,110.132440,-0.344191,0.938900,-2.727842,8,3,5,5,3,-2
3,-76.247143,-88.987143,-77.557500,-86.900000,-84.5200,-91.645000,-80.279444,-102.828000,-97.8275,12.740000,...,26.342135,0.896160,0.443730,0.495146,2,5,4,-3,-2,1
4,-75.950000,-88.482857,-74.288571,-86.510000,-91.5200,-93.597143,-84.063333,-104.362000,-81.3025,12.532857,...,-15.625616,0.963042,-0.269350,-0.279687,4,7,3,-3,1,4


In [110]:
# Essa Linha y_train = pd.read_csv("databases/Outdoor_y_train_90%.csv")
#y_train = db_train
#y_train = y_train.drop("idx", axis = 1)
#y_train = y_train.drop("lat", axis = 1)
#y_train = y_train.drop("lon", axis = 1)
# Essa linha y_train.head()
y_train_lon = pd.read_csv("Grid/grid_30.csv")
y_train_lon = y_train_lon.drop("lat", axis = 1)
y_train_lon.head()

,lon
0,-34.959850
1,-34.959549
2,-34.959248
3,-34.958948
4,-34.958647


In [111]:
# Essa linha X_test = pd.read_csv("Grid/grid_20.csv")
# Essa linha X_test.head()
X_test = pd.read_csv("databases/Outdoor_X_test_10%.csv")
X_test = X_test.drop("idx", axis = 1)
X_test = X_test.drop("lat", axis = 1)
X_test = X_test.drop("lon", axis = 1)
X_test.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delta_1_12,...,ang_3,cos_3,sin_3,tg_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23
0,-94.588571,-97.102857,-94.058571,-96.128333,-92.27500,-90.411429,-77.951667,-73.190,-94.7525,2.514286,...,94.650653,-0.081080,0.996708,-12.292874,6,5,1,1,5,4
1,-103.010000,-84.498571,-97.591429,-95.010000,-98.83750,-97.332857,-100.650000,-101.620,-94.4075,-18.511429,...,52.781288,0.604859,0.796332,1.316558,3,3,6,0,-3,-3
2,-95.202857,-86.280000,-93.030000,-53.330000,-84.96500,-81.080000,-90.826667,-94.342,-98.9125,-8.922857,...,65.551640,0.413873,0.910335,2.199551,4,2,6,2,-2,-4
3,-92.422619,-90.242143,-93.064643,-80.359286,-87.13750,-86.910952,-91.338333,-93.248,-97.2550,-2.180476,...,81.758935,0.143338,0.989674,6.904462,5,2,5,3,0,-3
4,-86.227143,-86.231429,-87.885714,-72.830000,-80.56625,-85.737143,-90.191667,-90.704,-97.6000,0.004286,...,72.359742,0.303040,0.952978,3.144731,5,2,5,3,0,-3


In [112]:
#reg_cso = RandomForestRegressor(n_estimators = 115, random_state = 42, criterion = 'mae')
#reg_cso.fit(X_train, y_train)
#y_pred_cso = reg_cso.predict(X_test)

# LightGBM 

In [113]:
# Splitting the dataset into the Training set and Test set
#from sklearn.model_selection import train_test_split
#x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [114]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [115]:
import lightgbm as lgb

In [116]:
d_train_lon = lgb.Dataset(X_train, label=y_train_lon)
d_train_lon

In [117]:
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['metric'] = 'mae'
params['sub_feature'] = 0.8
params['num_leaves'] = 28
#params['min_data'] = 50
#params['max_depth'] = 10
clf = lgb.train(params, d_train_lon, 10000)

In [118]:
#Prediction
y_pred_lon=clf.predict(X_test)
#y_pred_lon

In [119]:
X_test = pd.read_csv("databases/Outdoor_X_test_10%.csv")
X_test = X_test.drop("lat", axis = 1)
X_test = X_test.drop("lon", axis = 1)

In [120]:
y_pred_lon = pd.DataFrame({'lon': y_pred_lon})
#y_pred_lon

In [121]:
write_to_csv(X_test, y_pred_lon, "fingerprint_LightGBM_grid_20.csv")

In [124]:
parcialResult = pd.concat([X_test, y_pred_lon], axis=1)
parcialResult.head()

,idx,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,...,cos_3,sin_3,tg_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23,lon
0,7080,-94.588571,-97.102857,-94.058571,-96.128333,-92.27500,-90.411429,-77.951667,-73.190,-94.7525,...,-0.081080,0.996708,-12.292874,6,5,1,1,5,4,-34.950584
1,9438,-103.010000,-84.498571,-97.591429,-95.010000,-98.83750,-97.332857,-100.650000,-101.620,-94.4075,...,0.604859,0.796332,1.316558,3,3,6,0,-3,-3,-34.951302
2,8840,-95.202857,-86.280000,-93.030000,-53.330000,-84.96500,-81.080000,-90.826667,-94.342,-98.9125,...,0.413873,0.910335,2.199551,4,2,6,2,-2,-4,-34.950420
3,7629,-92.422619,-90.242143,-93.064643,-80.359286,-87.13750,-86.910952,-91.338333,-93.248,-97.2550,...,0.143338,0.989674,6.904462,5,2,5,3,0,-3,-34.951460
4,7988,-86.227143,-86.231429,-87.885714,-72.830000,-80.56625,-85.737143,-90.191667,-90.704,-97.6000,...,0.303040,0.952978,3.144731,5,2,5,3,0,-3,-34.949914


In [128]:
# Essa Linha y_train = pd.read_csv("databases/Outdoor_y_train_90%.csv")
#y_train = db_train
#y_train = y_train.drop("idx", axis = 1)
#y_train = y_train.drop("lat", axis = 1)
#y_train = y_train.drop("lon", axis = 1)
# Essa linha y_train.head()
y_train_lat = pd.read_csv("Grid/grid_30.csv")
y_train_lat = y_train_lat.drop("lon", axis = 1)
y_train_lat.head()

,lat
0,-8.05955
1,-8.05955
2,-8.05955
3,-8.05955
4,-8.05955


In [129]:
# Essa linha X_test = pd.read_csv("Grid/grid_20.csv")
# Essa linha X_test.head()
X_test = pd.read_csv("databases/Outdoor_X_test_10%.csv")
X_test = X_test.drop("idx", axis = 1)
X_test = X_test.drop("lat", axis = 1)
X_test = X_test.drop("lon", axis = 1)
X_test.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delta_1_12,...,ang_3,cos_3,sin_3,tg_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23
0,-94.588571,-97.102857,-94.058571,-96.128333,-92.27500,-90.411429,-77.951667,-73.190,-94.7525,2.514286,...,94.650653,-0.081080,0.996708,-12.292874,6,5,1,1,5,4
1,-103.010000,-84.498571,-97.591429,-95.010000,-98.83750,-97.332857,-100.650000,-101.620,-94.4075,-18.511429,...,52.781288,0.604859,0.796332,1.316558,3,3,6,0,-3,-3
2,-95.202857,-86.280000,-93.030000,-53.330000,-84.96500,-81.080000,-90.826667,-94.342,-98.9125,-8.922857,...,65.551640,0.413873,0.910335,2.199551,4,2,6,2,-2,-4
3,-92.422619,-90.242143,-93.064643,-80.359286,-87.13750,-86.910952,-91.338333,-93.248,-97.2550,-2.180476,...,81.758935,0.143338,0.989674,6.904462,5,2,5,3,0,-3
4,-86.227143,-86.231429,-87.885714,-72.830000,-80.56625,-85.737143,-90.191667,-90.704,-97.6000,0.004286,...,72.359742,0.303040,0.952978,3.144731,5,2,5,3,0,-3


In [130]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [131]:
import lightgbm as lgb

In [132]:
d_train_lat = lgb.Dataset(X_train, label=y_train_lat)
d_train_lat

In [133]:
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['metric'] = 'mae'
params['sub_feature'] = 0.8
params['num_leaves'] = 28
#params['min_data'] = 50
#params['max_depth'] = 10
clf = lgb.train(params, d_train_lat, 10000)

In [139]:
#Prediction
y_pred_lat=clf.predict(X_test)
#y_pred_lat
y_pred_lat = pd.DataFrame({'lat': y_pred_lat})
y_pred_lat.head()

,lat
0,-8.050560
1,-8.053089
2,-8.054150
3,-8.056325
4,-8.054176


In [140]:
write_to_csv(parcialResult, y_pred_lat, "fingerprint_LightGBM_grid_20.csv")

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators': list(range(100, 125)), 'criterion': ["mse", "mae"]}
random_forest = RandomForestRegressor()

grid = GridSearchCV(
    random_forest,
    parameters,
    cv = 10,
    scoring = 'neg_mean_squared_error',
    return_train_score = True,
    refit = True
)
grid.fit(X_train, y_train)
grid.best_params_

In [ ]:
Os parâmetros variados foram: o kernel, o parâmetro de penalidade C e o gamma. Os outros parâmetros ficaram com os valores
padrões da biblioteca. Os kernels testados foram sigmoid, linear, polinomial e RBF. 
O C variou em potências de 2, desde o valor de 2^−3 até o valor de 2^8. O gamma também variou em potências de 2,
os valores foram de 2^−5 até 2^3.

In [ ]:
Os melhores parâmetros para essa fase foram: kernel sigmoid, C = 256 e gamma = 4.

In [44]:
import math
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.utils import shuffle

In [45]:
X = X_train
#X.head()
#X = dataset.iloc[:, [0, 12]]
#y = dataset.iloc[:, 13]
X.head()

,dist_1,dist_2,dist_3,ang_1,cos_1,sin_1,tg_1,ang_2,cos_2,sin_2,...,ang_3,cos_3,sin_3,tg_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23
0,1.623132,0.504054,1.266725,158.234761,-0.928711,0.370804,-0.399268,-155.889163,-0.912757,-0.408503,...,98.686639,-0.151030,0.988529,-6.545237,7,2,5,5,2,-3
1,0.946625,0.892100,0.682786,-175.257525,-0.996576,-0.082677,0.082961,-83.307306,0.116544,-0.993186,...,56.915837,0.545870,0.837870,1.534924,4,4,3,0,1,1
2,0.813049,1.083588,0.677145,-162.727543,-0.954904,-0.296916,0.310938,-75.518359,0.250070,-0.968228,...,37.151782,0.797038,0.603929,0.757716,3,5,3,-2,0,2
3,0.827905,0.499070,1.382476,138.771078,-0.752082,0.659069,-0.876326,-31.683490,0.850962,-0.525226,...,59.893937,0.501602,0.865098,1.724670,4,2,6,2,-2,-4
4,0.702384,0.731350,1.042135,165.550108,-0.968366,0.249533,-0.257685,-59.863447,0.502063,-0.864831,...,52.394370,0.610223,0.792230,1.298263,3,3,4,0,-1,-1


In [46]:
y = y_train_rssi_1_1
y.head()

,rssi_1_1
0,-87.185714
1,-85.330000
2,-90.572857
3,-96.702857
4,-91.864286


In [47]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

In [48]:
seed = 42
X, y = shuffle(X, y, random_state=seed)

In [60]:
def svr_model(X, y):
    gsc = GridSearchCV(
        estimator=SVR(),
        param_grid={
            'C': [0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32, 64, 128, 256],
            'epsilon': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
            'gamma': [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8],
            'kernel': ['sigmoid', 'linear','rbf']
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    best_svr = SVR(kernel=best_params["rbf"], C=best_params["C"], epsilon=best_params["epsilon"], gamma=best_params["gamma"],
                   coef0=0.1, shrinking=True,
                   tol=0.001, cache_size=200, verbose=False, max_iter=-1)

    scoring = {
               'abs_error': 'neg_mean_absolute_error',
               'squared_error': 'neg_mean_squared_error'}

    scores = cross_validate(best_svr, X, y, cv=10, scoring=scoring, return_train_score=True)
    return "MAE :", abs(scores['test_abs_error'].mean()), "| RMSE :", math.sqrt(abs(scores['test_squared_error'].mean())), "| best_svr :", best_svr

In [61]:
print(svr_model(X,y))

C:\Users\DOUGLAS.SILVA\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DOUGLAS.SILVA\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DOUGLAS.SILVA\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DOUGLAS.SILVA\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

('MAE :', 2.032766183511892, '| RMSE :', 3.270037850352753, '| best_svr :', SVR(C=64, cache_size=200, coef0=0.1, degree=3, epsilon=0.5, gamma=8,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))


In [ ]:
('MAE :', 2.083546343441733, '| RMSE :', 3.3166580120758025, '| best_svr :', SVR(C=100, cache_size=200, coef0=0.1, degree=3, epsilon=0.5, gamma=5,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))

In [ ]:
('MAE :', 2.032766183511892, '| RMSE :', 3.270037850352753, '| best_svr :', SVR(C=64, cache_size=200, coef0=0.1, degree=3, epsilon=0.5, gamma=8,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))

# Grid Search LightGBM:

In [63]:
import math
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.utils import shuffle
import lightgbm as lgb

In [64]:
X_LightGBM = X_train
X_LightGBM.head()

,dist_1,dist_2,dist_3,ang_1,cos_1,sin_1,tg_1,ang_2,cos_2,sin_2,...,ang_3,cos_3,sin_3,tg_3,delay_1,delay_2,delay_3,delay_12,delay_13,delay_23
0,1.623132,0.504054,1.266725,158.234761,-0.928711,0.370804,-0.399268,-155.889163,-0.912757,-0.408503,...,98.686639,-0.151030,0.988529,-6.545237,7,2,5,5,2,-3
1,0.946625,0.892100,0.682786,-175.257525,-0.996576,-0.082677,0.082961,-83.307306,0.116544,-0.993186,...,56.915837,0.545870,0.837870,1.534924,4,4,3,0,1,1
2,0.813049,1.083588,0.677145,-162.727543,-0.954904,-0.296916,0.310938,-75.518359,0.250070,-0.968228,...,37.151782,0.797038,0.603929,0.757716,3,5,3,-2,0,2
3,0.827905,0.499070,1.382476,138.771078,-0.752082,0.659069,-0.876326,-31.683490,0.850962,-0.525226,...,59.893937,0.501602,0.865098,1.724670,4,2,6,2,-2,-4
4,0.702384,0.731350,1.042135,165.550108,-0.968366,0.249533,-0.257685,-59.863447,0.502063,-0.864831,...,52.394370,0.610223,0.792230,1.298263,3,3,4,0,-1,-1


In [65]:
y_LightGBM = y_train_rssi_1_1
y_LightGBM.head()

,rssi_1_1
0,-87.185714
1,-85.330000
2,-90.572857
3,-96.702857
4,-91.864286


In [66]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_LightGBM = scaler.fit_transform(X_LightGBM)

In [67]:
seed = 42
X_LightGBM, y_LightGBM = shuffle(X_LightGBM, y_LightGBM, random_state=seed)

In [70]:
def LightGBM_model(X, y):
    gsc = GridSearchCV(
        estimator=lgb(),
        param_grid={
            'learning_rate': [1.0],
            'boosting_type': ['gbdt', 'goss'],
            'metric': ['mae'],
            'sub_feature': [0.8],
            'num_leaves': [255],
            'min_data': [1],
            'max_depth': [10]
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    best_svr = SVR(learning_rate=best_params["learning_rate"], boosting_type=best_params["boosting_type"], metric=best_params["metric"], sub_feature=best_params["sub_feature"],
                   num_leaves=best_params["num_leaves"], min_data=best_params["min_data"], max_depth=best_params["max_depth"],
                   coef0=0.1, shrinking=True,
                   tol=0.001, cache_size=200, verbose=False, max_iter=-1)

    scoring = {
               'abs_error': 'neg_mean_absolute_error',
               'squared_error': 'neg_mean_squared_error'}

    scores = cross_validate(best_svr, X, y, cv=10, scoring=scoring, return_train_score=True)
    return "MAE :", abs(scores['test_abs_error'].mean()), "| RMSE :", math.sqrt(abs(scores['test_squared_error'].mean())), "| best_svr :", best_svr

In [72]:
print(LightGBM_model(X_LightGBM ,y_LightGBM ))

TypeError: 'module' object is not callable

In [ ]:
params['learning_rate'] = 1.0
params['boosting_type'] = 'gbdt'#'gbdt'
params['metric'] = 'mae'
params['sub_feature'] = 0.8
params['num_leaves'] = 255
params['min_data'] = 1
#params['max_depth'] = 10

In [80]:
# Set params
# Scores ~0.784 (without tuning and early stopping)
params = {'boosting_type': 'gbdt',
          'num_leaves': 64,
          'learning_rate': 0.05,
          'metric' : 'mae',}

# Create parameters to search
gridParams = {
    'learning_rate': [0.005, 1.0],
    'boosting_type' : ['gbdt','goss'],
    'metric':  ['mae'],
    'sub_feature': [0.8],
    'num_leaves': [6,8,12,16,255],
    'min_data': [1],
    'max_depth': [-1,10],
    }

# Create classifier to use. Note that parameters have to be input manually
# not as a dict!
mdl = lgb.LGBMClassifier(boosting_type= 'gbdt',
          silent = True,
          max_depth = params['max_depth'],
          
# To view the default model params:
mdl.get_params().keys()

# Create the grid
grid = GridSearchCV(mdl, gridParams, verbose=0,cv=4)
# Run the grid
grid.fit(X_LightGBM, y_LightGBM)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)

# Using parameters already set above, replace in the best from the grid search
params['colsample_bytree'] = grid.best_params_['colsample_bytree']
params['learning_rate'] = grid.best_params_['learning_rate']
# params['max_bin'] = grid.best_params_['max_bin']
params['num_leaves'] = grid.best_params_['num_leaves']
params['reg_alpha'] = grid.best_params_['reg_alpha']
params['reg_lambda'] = grid.best_params_['reg_lambda']
params['subsample'] = grid.best_params_['subsample']
# params['subsample_for_bin'] = grid.best_params_['subsample_for_bin']

print('Fitting with params: ')
print(params)


SyntaxError: invalid syntax (<ipython-input-80-30463c55fbeb>, line 29)

In [75]:
# Set params
# Scores ~0.784 (without tuning and early stopping)
params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary',
          'nthread': 3, # Updated from nthread
          'num_leaves': 64,
          'learning_rate': 0.05,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          'min_split_gain': 0.5,
          'min_child_weight': 1,
          'min_child_samples': 5,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error'}

# Create parameters to search
gridParams = {
    'learning_rate': [0.005],
    'n_estimators': [40],
    'num_leaves': [6,8,12,16],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }

# Create classifier to use. Note that parameters have to be input manually
# not as a dict!
mdl = lgb.LGBMClassifier(boosting_type= 'gbdt',
          objective = 'binary',
          n_jobs = 3, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'],
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'],
          subsample_freq = params['subsample_freq'],
          min_split_gain = params['min_split_gain'],
          min_child_weight = params['min_child_weight'],
          min_child_samples = params['min_child_samples'],
          scale_pos_weight = params['scale_pos_weight'])

# To view the default model params:
mdl.get_params().keys()

# Create the grid
grid = GridSearchCV(mdl, gridParams,
                    verbose=0,
                    cv=4,
                    n_jobs=2)
# Run the grid
grid.fit(X_LightGBM, y_LightGBM)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)

# Using parameters already set above, replace in the best from the grid search
params['colsample_bytree'] = grid.best_params_['colsample_bytree']
params['learning_rate'] = grid.best_params_['learning_rate']
# params['max_bin'] = grid.best_params_['max_bin']
params['num_leaves'] = grid.best_params_['num_leaves']
params['reg_alpha'] = grid.best_params_['reg_alpha']
params['reg_lambda'] = grid.best_params_['reg_lambda']
params['subsample'] = grid.best_params_['subsample']
# params['subsample_for_bin'] = grid.best_params_['subsample_for_bin']

print('Fitting with params: ')
print(params)


ValueError: Unknown label type: 'continuous'

In [ ]:

# Kit k models with early-stopping on different training/validation splits
k = 4
predsValid = 0
predsTrain = 0
predsTest = 0
for i in range(0, k):
    print('Fitting model', k)

    # Prepare the data set for fold
    trainData, validData = train_test_split(train,
                                            test_size=0.4,
                                            stratify=train.Survived)
    trainDataL, trainLabels, trainIDs, trainData = prepLGB(trainData,
                                                     classCol='Survived',
                                                     IDCol='PassengerId',
                                                     fDrop=fDrop)
    validDataL, validLabels, validIDs, validData = prepLGB(validData,
                                                     classCol='Survived',
                                                     IDCol='PassengerId',
                                                     fDrop=fDrop)
    # Train
    gbm = lgb.train(params,
                    trainDataL,
                    100000,
                    valid_sets=[trainDataL, validDataL],
                    early_stopping_rounds=50,
                    verbose_eval=4)